In [ ]:
# Compile mediapipe
import os
import json
from os.path import exists, join, basename, splitext

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

class GoogleDriveDatabase:
  def __init__(self, drive, DATABASE_GID:str):
    assert isinstance(DATABASE_GID, str)
    self.folders = {}
    self.drive = drive
    self.database_gid = DATABASE_GID
    folder_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(DATABASE_GID)}).GetList()
    for file in folder_list:
      if file['mimeType'] == "application/vnd.google-apps.folder":
        self.folders[file['title']] = file['id']
    self.createFolder("TRASH")
    print("{} folders loaded".format(len(self.folders.keys())))
  def upload(self, filename, character, fileType):
    assert isinstance(filename, str)
    assert isinstance(character, str)
    assert isinstance(fileType, str)
    FILETYPE_MIME_MAP = {
        "jpeg" : "image/jpeg",
        "json" : "application/json",
        "zip" : "application/zip"
    }
    assert fileType in FILETYPE_MIME_MAP.keys(), "fileType must be one of the following: {}".format(fileType)
    assert os.path.isfile(filename), "{} does not exist as a file".format(filename)
    assert self.checkFolder(character), "{} is not a valid character. Pick from list: \n{}".format(character, tuple(self.folders.keys()))
    file = self.drive.CreateFile({
        "title" :  os.path.split(filename)[1],
        "mimeType" : FILETYPE_MIME_MAP[fileType],
        "parents" : [{"id" : self.folders[character]}]
    })
    file.SetContentFile(filename)
    file.Upload()
    os.remove(filename)
    print("uploaded and deleted {}".format(filename))

  @staticmethod
  def allImagesToJPG(fileName) -> str:
    assert isinstance(fileName, str)
    assert any(extension for extension in GoogleDriveDatabase.IMAGE_EXTENSIONS()), "{} is not a valid image".format(fileName)
    if ".jpg" in fileName:
      return fileName
    return fileName.split(".")[0] + ".jpg"
    
  @staticmethod
  def IMAGE_EXTENSIONS() -> list:
    return [".jpg", ".jpeg", ".png"]

  @staticmethod
  def FILE_EXTENSIONS() -> list:
    return [".jpg", ".jpeg", ".png", ".zip", ".json"]

  def getFiles(self, character) -> list:
    return [
      x for x in self.drive.ListFile({'q': "'{}' in parents and trashed=false".format(self.folders[character])}).GetList()
      if x['mimeType'] != "application/vnd.google-apps.folder" 
    ]
  def download_file_name(self, file_name):
      if not any(extension in file_name for extension in GoogleDriveDatabase.FILE_EXTENSIONS()):
        file_name += ".jpg"
      return GoogleDriveDatabase.allImagesToJPG(file_name)

  def download_file(self, file, folder:str, **kwargs) -> str:
      check_already_exist = kwargs.get("check_local", False)
      file_name = os.path.join(folder, file['title'])
      file_name = self.download_file_name(file_name)
      if check_already_exist:
        local_files = [os.path.join(folder, file) for file in os.listdir(folder)]
        if file_name in local_files:
          return file_name
      file.GetContentFile(file_name)
      print("downloaded", file_name)
      return file_name

  def download(self, character:str,folder:str):
    file_list = self.getFiles(character)
    os.makedirs(folder, exist_ok=True)
    returnlist = []
    
    for file in file_list:
      returnlist.append(self.download_file(file, folder, check_local=True))
    return tuple(returnlist)

  def checkFolder(self, name):
    assert isinstance(name, str)
    return name in self.folders.keys()

  def createFolder(self, name, exist_ok=True):
    if self.checkFolder(name):
      if exist_ok:
        return True
      raise Exception("Folder {} already exists".format(name))
    self.drive.CreateFile({
        "title" : name,
        "mimeType" : "application/vnd.google-apps.folder",
        "parents" : [{"id" : self.database_gid}]
    }).Upload()
  def move_file(self, file_obj, newFolder):
    assert isinstance(newFolder, str)
    assert self.checkFolder(newFolder)
    files = self.drive.auth.service.files()
    file = files.get(fileId=file_obj['id'], fields ='parents').execute()
    prev_parents = ','.join(p['id'] for p in file.get('parents'))
    file = files.update(
        fileId = file_obj['id'],
        addParents = self.folders[newFolder],
        removeParents = prev_parents,
        fields = 'id, parents',
    ).execute()
    return file
  def trash(self, file_obj):
    self.move_file(file_obj, "TRASH")
    print("Sent {} to trash".format(file_obj['title']))

auth.authenticate_user() # Google auth stuff, make sure to sign in with your ucsb account
gauth = GoogleAuth() # Google auth stuff
gauth.credentials = GoogleCredentials.get_application_default() # Google auth stuff
drive = GoogleDrive(gauth) # Google auth stuff


DOWNLOAD_DATABASE = "1RqiJwO6i1KJx54tRC3QJcHWjiooCLJNC"
UPLOAD_DATABASE = "1wYWIJFfm2la6KrimpC5mWFBpyITcUT6U"
download_database = GoogleDriveDatabase(drive, DOWNLOAD_DATABASE)
upload_database = GoogleDriveDatabase(drive,UPLOAD_DATABASE)
CHARACTERS = ("A", "B", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y")
[upload_database.createFolder(x,exist_ok=True) for x in CHARACTERS]
git_repo_url = 'https://github.com/AriAlavi/SigNN.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  !git clone -q --depth 1 $git_repo_url
  !sudo apt install curl
  !curl https://bazel.build/bazel-release.pub.gpg | sudo apt-key add -
  !echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
  !sudo apt update && sudo apt install bazel-3.3.0
  !sudo apt-get install libopencv-core-dev libopencv-highgui-dev \
                        libopencv-calib3d-dev libopencv-features2d-dev \
                        libopencv-imgproc-dev libopencv-video-dev
  !cd {project_name} && bazel-3.3.0 build -c opt --define MEDIAPIPE_DISABLE_GPU=1 mediapipe/examples/desktop/multi_hand_tracking:multi_hand_tracking_cpu 
else:
  !git config --global user.email "none@gmail.com"
  !git config --global user.name "Google colab"
  !cd {project_name} && git stash
  !cd {project_name} && git pull
  !cd {project_name} && bazel-3.3.0 build -c opt --define MEDIAPIPE_DISABLE_GPU=1 mediapipe/examples/desktop/multi_hand_tracking:multi_hand_tracking_cpu 

26 folders loaded
25 folders loaded
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3199  100  3199    0     0  15681      0 --:--:-- --:--:-- --:--:-- 15681
OK
deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [95.7 kB]
Get:3 https://storage.googleapis.com/bazel-apt stable InRelease [2,256 B]
Ign:4 https://developer.download.nvidia.com/comput

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from datetime import datetime



import uuid
import os
import time





BASE_RESOLUTION = (1920, 1920)

def createVideoFromDirectory(directory, output_path):
  assert isinstance(directory, str)
  assert isinstance(output_path, str)
  !ffmpeg -framerate 1 -pattern_type glob -i "{directory}/*.jpg" -c:v libx264 -r 1 -pix_fmt yuv420p {output_path} -loglevel error -y
  return output_path

def getImagesInFolder(absolutePath):
    assert isinstance(absolutePath, str)
    image_names = [os.path.join(absolutePath, img) for img in os.listdir(absolutePath) if img.endswith(".jpg") or img.endswith(".jpeg") or img.endswith(".png")]
    return image_names
  
def runMediapipe(input_video):
  assert os.path.isfile(input_video), "{} does not exist".format(input_video)
  !cd SigNN && sudo GLOG_logtostderr=0 bazel-bin/mediapipe/examples/desktop/multi_hand_tracking/multi_hand_tracking_cpu --calculator_graph_config_file=mediapipe/graphs/hand_tracking/multi_hand_tracking_desktop_logger.pbtxt --input_video_path={input_video} --render_video=false

def modifyMediapipeLoggerOutput(path, filename, mediapipe_directory):
  assert isinstance(path, str)
  assert isinstance(filename, str)
  assert filename.split(".")[-1] == "json"
  assert isinstance(mediapipe_directory, str)
  os.makedirs(path, exist_ok=True)
  pbtxt_name = os.path.join(mediapipe_directory, "mediapipe/graphs/hand_tracking/multi_hand_tracking_desktop_logger.pbtxt")
  pbtxt_file = open(pbtxt_name, "r+b")
  pbtxt_content = pbtxt_file.readlines()
  line_count = 0
  for line in pbtxt_content:
    if "CoordinateLoggerCalculatorOptions" in str(line):
      pbtxt_content[line_count + 2] = '      logger_path: "{}"\n'.format(path).encode()
      pbtxt_content[line_count + 3] = '      filename: "{}"\n'.format(filename).encode()
      break
    line_count += 1
  pbtxt_file.seek(0)
  pbtxt_file.truncate()
  pbtxt_file.write(''.join([x.decode() for x in pbtxt_content]).encode())
  pbtxt_file.close()

class InvalidMediapipeOutput(Exception):
  pass

def Mediapipe(input_directory, output_directory):
  VIDEO_NAME = "out.mp4"
  os.makedirs(input_directory, exist_ok=True)
  os.makedirs(output_directory, exist_ok=True)
  print("Setting mediapipe logger output")
  modifyMediapipeLoggerOutput(output_directory, "output.json", "/content/SigNN/")
  video_path = os.path.join(input_directory, VIDEO_NAME)
  OUTPUT_FILE = os.path.join(output_directory, "output.json")
  image_raw_names = getImagesInFolder(input_directory)
  print("Creating videos from images")
  createVideoFromDirectory(input_directory, video_path)
  print("Running mediapipe on {} images".format(len(getImagesInFolder(input_directory))))
  runMediapipe(video_path)
  output_file_obj = open(OUTPUT_FILE, "r")
  output_file_single_list = json.load(output_file_obj)
  output_file_obj.close()
  if len(output_file_single_list) != len(image_raw_names):
    raise InvalidMediapipeOutput("There were {} coordinates created but there were {} images fed in".format(len(output_file_single_list), len(image_raw_names)))
  i = 0
  json_files = []
  for coordinates in output_file_single_list:
    image_full_path = image_raw_names[i]
    image_name = os.path.split(image_full_path)[1]
    json_name = image_name.split(".")[0] + ".json"
    json_path = os.path.join(output_directory, json_name)
    json_file = open(json_path, "w")
    json.dump(coordinates, json_file)
    json_file.close()
    json_files.append(json_path)
    i += 1
  !rm -rf {input_directory}
  return json_files



# def download_characters(): # Only downloads characters
#   BASE_DIR = "images/"
#   results = {}
#   for c in CHARACTERS:
#     results[c] = download_database.download(c, os.path.join(BASE_DIR, c + "/"))
#     print("{} pictures downloaded for {}".format(len(results[c]), c))
#   return results
    
def DownloadCharacterAndUploadJson(character, max_count, **kwargs):
  assert isinstance(character, str)
  assert isinstance(max_count, int)
  ACCEPT_KWARGS = ["only_get_pictures", "prefed_pictures"]
  only_get_pictures = kwargs.get("only_get_pictures", False)
  prefed_pictures = kwargs.get("prefed_pictures", [])
  assert isinstance(only_get_pictures, bool)
  assert isinstance(prefed_pictures, list)
  for x in kwargs.keys():
    if x not in ACCEPT_KWARGS:
      raise Exception("{} is not a valid kwargs argument".format(x))

  already_analyzed_names = set() # Names of all already analyzed items
  pictures_to_analyze = [] # Pictures that yet do not have a json file
  pictures_to_analyze_names = [] # Names of pictures that yet do not have a json file
  BASE_FOLDER = "images/" # Directory where all images are
  character_folder = os.path.join(BASE_FOLDER, character + "/") # Folder in which the images for this character are in (i.e. images/A)
  os.makedirs(character_folder, exist_ok=True) # Make the character folder if it doesn't exist
  if only_get_pictures: # Uneeded unless specifically asked for in kwargs
    pictures_to_analyze_references = [] # References of all pictures to analyze
  strip_name = lambda x: x.split("/")[-1].split(".")[0] # Function to remove extensions (i.e. .json .jpg) from filenames

  if not prefed_pictures:
    character_jsons_original = upload_database.getFiles(character) # All .json files created for this character
    character_jsons_names = [strip_name(x['title']) for x in character_jsons_original] # Name of all .json files already created for this character
    picture_references = download_database.getFiles(character) # All current pictures for this character
    picture_reference_names = [strip_name(x['title']) for x in picture_references] # Name of all pictures
    print("There are {} images for character {}".format(len(picture_references), character))
    print("There are {} json files already made for character {}".format(len(character_jsons_names), character))
    for json_original in character_jsons_original: # For every json file on the google drive...
      if strip_name(json_original['title']) not in picture_reference_names: # ...if the json file is not in the list of pictures...
        print("{} deleted because no image was found to relate to it".format(json_original['title']))
        upload_database.trash(json_original) # ...then delete the file
  else:
    picture_references = prefed_pictures
    print("{} pictures pre-fed".format(len(prefed_pictures)))

  for picture in picture_references: # For every picture on google drive...
    picture_name = strip_name(picture['title'])
    if not prefed_pictures and picture_name in character_jsons_names: # ...if that picture already has a json uploaded...
      already_analyzed_names.add(download_database.download_file_name(picture['title'])) # ...add its name to the list of already analyzed pictures
      continue
    elif max_count != 0:
      pictures_to_analyze.append(download_database.download_file(picture, character_folder, check_local=True)) # ...else download it
      pictures_to_analyze_names.append(picture_name)
      if only_get_pictures:
        pictures_to_analyze_references.append(picture)
      max_count -= 1
    else:
      break
  if only_get_pictures:
    return pictures_to_analyze_references

  local_duplicates = [os.remove(os.path.join(character_folder, x)) for x in os.listdir(character_folder) if os.path.isfile(os.path.join(character_folder, x)) and strip_name(x) not in pictures_to_analyze_names] # if a picture is not supposed to be analyzed list and exists in the local directory, delete it so that mediapipe doesn't waste time running it
  print("{} extra local files detected".format(len(local_duplicates)))
  if len(os.listdir(character_folder)) == 0:
    print("{} has no images to process".format(character))
    return []

  BASE_IMAGES_DIR = "/content/images/"
  BASE_JSON_DIR = "/content/json/"
  json_created = Mediapipe(os.path.join(BASE_IMAGES_DIR, character), os.path.join(BASE_JSON_DIR, character)) # runs mediapipe on all images in a given path and outputs each image to an output path
  for json_file in json_created: # For every local json file created...
    upload_database.upload(json_file, character, "json") #...upload it to google drive
  return pictures_to_analyze

def DownloadCharacterAndUploadJsonRecursively(character, max_count=100, prefed_set=[]):
  print("\n")
  if len(prefed_set) == 1: # Recursive base case, if there is one pictures that is causing problems
    download_database.trash(prefed_set[0]) # Then delete it
    return [] # There is no valid picture
  error = None
  try:
    return DownloadCharacterAndUploadJson(character, max_count, prefed_pictures=prefed_set) # Try to run mediaipe on the set
  except InvalidMediapipeOutput as e: # If there is a picture casuing problems
    error = e
  except:
    time.sleep(60 * 5)
    gauth.Refresh()
    return DownloadCharacterAndUploadJson(character, max_count, prefed_pictures=prefed_set)
  if len(prefed_set) == 0: # And there is no history of pictures casuing problems
    prefed_set = DownloadCharacterAndUploadJson(character, max_count, only_get_pictures=True) # Get a list of the suspect pictures
  print("Problems with mediapipe pictures: {}. Splitting list of {} in half.".format(error, len(prefed_set)))
  half_count = int(len(prefed_set) / 2)
  first_half = prefed_set[:half_count] # Divide the list of suspect pictures in half
  second_half = prefed_set[half_count:] # This is the second half
  if len(first_half) > 0:
    files_created = DownloadCharacterAndUploadJsonRecursively(character, max_count, first_half) # Recrusive call 1 (Hope this list does not have picture causing problems)
  if len(second_half) > 0:
    files_created += DownloadCharacterAndUploadJsonRecursively(character, max_count, second_half) # Recursive call 2 (Hope htis list does not have picture causing problems)
  return files_created # Return all valid pictures
    

max_count = 100

while True:
  for c in CHARACTERS:
    if len(DownloadCharacterAndUploadJsonRecursively(c, max_count)) > max(max_count, 100) / 2:
      gauth.Refresh()
  time.sleep(60 * 5)
  gauth.Refresh()

